In [2]:
# importing libraries
from bs4 import BeautifulSoup
import requests
import nltk; from nltk.corpus import stopwords; from nltk.stem import PorterStemmer
import heapq

#==========================================================================================================================
# Part related to the crawling of wikipedia
#==========================================================================================================================

# doing the request to the web page with the films
request = requests.get("https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html")
soup = BeautifulSoup(request.text, 'html.parser')

# extract all the links that are into the page and insert all the links into a list
film_list = soup.select("a")
for i in range(len(film_list)):
    film_list[i] = film_list[i].get("href")

# for each link that is into the list created previously extract the HTML code related to that link and insert it into a
# file created at the moment into the position specified by the url_file variable
for i in range(len(film_list)):
    url_file = "C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles3\\article_" + str(i+20001) + ".html"
    file = open(url_file, "w", encoding="utf-8")
    file.write(str(requests.get(film_list[i]).text))
    file.close()

#==========================================================================================================================
# That part extract all needed informations from the crawled pages
#==========================================================================================================================

# extracting data from crawled HTML pages and transform it into a film.tsv file with the extracted informations as specified
# into the requests
for i in range(len(film_list)):
    # inizializing variables
    flag = 0; 
    director = "NA"; producer = "NA"; writer = "NA"; starring = "NA"; music = "NA"; release_date = "NA"; runtime = "NA"; 
    country = "NA"; language = "NA"; budget = "NA"
    
    # creating url, open file and parsing it
    url = "C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\article_" + str(i+1) + ".html"
    fileIn = open(url, "r", encoding="utf-8")
    fileOut = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\film" + str(i+1) + ".tsv", "a", encoding="utf-8")
    film = BeautifulSoup(fileIn.read(), "html.parser")
    
    # extract title of the page and the film
    list_h1 = film.select("h1")
    for h1 in list_h1:
        if h1.get("id") == "firstHeading":
            fTitle = h1.text
            
    # this part trys to extract the intro and plot of the film, if they aren't into the page they are set to NA
    try:
        intro = film.select("p")[0].text
        plot = film.select("p")[1].text
    except:
        intro = "NA"; plot = "NA"
    finally:
    
        # search and then extract the required values from the infobox table related to the film
        fTables = film.select("table")
        for table in fTables:
            if table.get("class") != None:
                if "infobox" in table.get("class"):
                    rows = table.select("tr")
                    for row in rows:
                        for el in row.select("th"):
                            if "class" in el.attrs.keys():
                                if "summary" in el.attrs["class"]:
                                    flag = 1
                                    summary = el.text
                                else:
                                    summary = "NA"
                            if el.text == "Directed by":
                                director = row.select("td")[pos].text
                            if el.text == "Produced by":
                                producer = row.select("td")[pos].text
                            if el.text == "Written by":
                                writer = row.select("td")[pos].text
                            if el.text == "Starring":
                                starring = row.select("td")[pos].text
                            if el.text == "Music by":
                                music = row.select("td")[pos].text
                            if el.text == "Release date":
                                release_date = row.select("td")[pos].text
                            if el.text == "Running time":
                                runtime = row.select("td")[pos].text
                            if el.text == "Country":
                                country = row.select("td")[pos].text
                            if el.text == "Language":
                                language = row.select("td")[pos].text
                            if el.text == "Budget":
                                budget = row.select("td")[pos].text
                            pos += 1
                        pos = 0

        # write into the result file all the important data extracted according to the format of a tsv file
        if flag == 0: summary = "NA"
        fileOut.write(fTitle + " \t " + intro + " \t " + plot + " \t " + summary + " \t " + director + " \t " + producer + " \t " + 
                      writer + " \t " + starring + " \t " + music + " \t " + release_date + " \t " + runtime + " \t " + country +
                      " \t " + language + " \t " + budget)
    
    # close the files that aren't used
    fileIn.close(); fileOut.close()

#==========================================================================================================================
# This park makes the indexing of the extracted informations 
#==========================================================================================================================

# declarating global variables
rev_index_App = {}; rev_index = {}
commonWords = set(stopwords.words("english"))

# does a first reverse indexing, at the end the result is represented by a dictionary with words as a key and the value
# associated is a list of integers where each integer represent the number of the page where the word compares. Ex:
# {"word_1": [1, 2, 4, 5], "word_2": [2, 5, 6] ...}
# so the word_a is into the first, second, fourth and fifth pages, the word_2 is into second, fifth and sixth page and so on
for i in range(1, 10000):
    # opening files and variables
    fileIn = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\film" + str(i) + ".tsv", encoding="utf-8")
    intro_plot = []
    fileText = fileIn.read()
    
    # extract the summary, intro and the plot of each film article
    for j in range(3):
        intro_plot.append(fileText[:fileText.find("\t")].strip())
        fileText = fileText[fileText.find("\t") + 2:]

    # first removes the summary because it isn't required for the indexing of the words
    # then does a parsing of the string represented by the el variable transforming it into a list of words then
    # controls if the word is "legit" and insert it into a list that will be used after to construct a first result
    # dictionary as exposed into the comment that begins on the row 10
    intro_plot.pop(0)
    for el in intro_plot:
        listaParole = nltk.word_tokenize(el); listaParoleAcc = []
        for j in range(len(listaParole)):
            if listaParole[j].isalpha() == True and listaParole[j].lower() not in commonWords:
                listaParoleAcc.append(listaParole[j].lower())
        for word in listaParoleAcc:
            if word not in rev_index_App.keys():
                rev_index_App[word] = [i]
            else:
                if i not in rev_index_App[word]:
                    rev_index_App[word].append(i)
    fileIn.close()

# this part creates a matrix with 2 columns and 10000 rows, each row contains the id of the word and the word associated
# Ex: [[1, word_1], [2, word_2], [3, word_3] ...]
dict_id = []
for i in range(len(rev_index_App.keys())):
    dict_id.append([i+1, list(rev_index_App.keys())[i]])

# this part creates a new dictionary where the keys aren't the words but their index
# Ex: {1: [1, 2, 3, 4], 2: [2, 4, 6, 7] ...}
value = list(rev_index_App.values())
for i in range(len(rev_index_App.keys())):
    rev_index[i+1] = value[i]

# writes into two different files the matrix with the "conversion table" from an id to his word and the reverse index
# dictionary
fileIds = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\vocabulary.txt", "w", encoding="utf-8")
fileIds.write(str(dict_id))
fileIds.close()
fileDict = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles\\reverseIndex.txt", "w", encoding="utf-8")
fileDict.write(str(rev_index))
fileDict.close()

#==========================================================================================================================
# This part makes a film research using weighted scores 
#==========================================================================================================================

# this function controls if at least one word of the query matches with the words contained into an element of the film, if 
# this happen returnd a positive score, otherwise it returns 0
def assPunt(lisParAcc, lisParQuery, i):
    for q in lisParQuery:
        if q.lower() in lisParAcc:
            if 1 <= i <= 3:
                return 19
            else:
                return 4
    return 0

# creation of global variables
firstRes = {}; res = []; commonWords = set(stopwords.words("english"))

# handles the query and makes the reseach with the score
query = input("Inserisci le parole chiave della tua ricerca: ")
query = query.split(" ")
for i in range(10000):
    fileIn = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles1\\film" + str(i+1) + ".tsv", encoding="utf-8")
    info = []
    fileText = fileIn.read()
    tot = 0

    while fileText.find("\t") != -1:
        info.append(fileText[:fileText.find("\t")].strip())
        fileText = fileText[fileText.find("\t") + 2:]
    info.append(fileText.strip())

    # gives a score to the document and creates a dictionary with the number of the documents with highter score
    for el in info:
        cont = 1
        listaParole = nltk.word_tokenize(el); listaParoleAcc = []
        for word in listaParole:
            word = word.lower()
            if word.isalpha() == True and word not in commonWords and PorterStemmer().stem(word) not in listaParoleAcc:
                listaParoleAcc.append(PorterStemmer().stem(word))
        tot += assPunt(listaParoleAcc, query, cont)
        cont += 1

    if len(firstRes) < 10:
        firstRes[i+1] = round(tot/101, 4)
    else:
        if round(tot/101, 4) > min(firstRes.values()):
            for key in firstRes.keys():
                if firstRes[key] == min(firstRes.values()):
                    remKey = key
            firstRes.pop(remKey)
            firstRes[i+1] = round(tot/101, 4)

# creates the heap tree and extract the values from the tree (in practical it makes an heapSort)
heap = list(firstRes.values())
heapq.heapify(heap)
for i in range(len(firstRes)):
    res.insert(0, heapq.heappop(heap))

# search the number of the document with a certain scores and then print the required elements(title, intro, link, and score)
for el in res:
    for key, value in firstRes.items():
        if value == el:
            fileIn = open("C:\\Users\\asus\\Desktop\\Algoritmic methods for data science\\ADM-HM3\\articles1\\film" + str(key) + ".tsv", encoding="utf-8")
            fileText = fileIn.read()
            s = ""
            for j in range(2):
                s += fileText[:fileText.find("\t")].strip() + "\t"
                fileText = fileText[fileText.find("\t") + 2:]
            s += film_list[key-1] + "\t" + str(value)
            print(s)